# Performance Evaluation of BERT model in a Blue Amazon QA dataset

This code is about evaluating the performance of the model: bert-base-cased-squad-v1.1-portuguese (BERT model trained for brazilian portuguese SQuAD) for the dataset created by the authors of Q&A about Amazônia Azul (Exclusive economic zone of Brazil).

This is part of the work: *Interpretability of Attention Mechanisms in a Portuguese-Based Question Answering System\\about the Blue Amazon*
published at Encontro Nacional de Inteligência Artificial e Computacional 2021  (ENIAC 2021).

Check the github at: https://github.com/C4AI/blab-qa-viz



##Installing the main libraries 

In [1]:
!pip install transformers
!pip install rouge_score

     |████████████████████████████████| 3.1 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 30.7 MB/s 
     |████████████████████████████████| 3.3 MB 41.9 MB/s 
     |████████████████████████████████| 61 kB 542 kB/s 
     |████████████████████████████████| 596 kB 48.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Checking Colab GPU specs

In [2]:
!nvidia-smi

Tue Dec  7 02:39:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Downloading punkt from nltk

This will be important for the metrics part

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

##Loading bert-base-cased-squad-v1.1-portuguese transformers model

Check the model at: https://huggingface.co/pierreguillou/bert-base-cased-squad-v1.1-portuguese

In [4]:
from transformers import pipeline

# Loading the model
qa_pipeline = pipeline(
    "question-answering",
    model="pierreguillou/bert-base-cased-squad-v1.1-portuguese",
    tokenizer="pierreguillou/bert-base-cased-squad-v1.1-portuguese"
)

# Doing one prediction to check answer
predictions = qa_pipeline({
    'context': "O rio Nilo é o rio mais comprido do mundo",
    'question': "Qual é o rio mais comprido do mundo?"
})

print(predictions)

Downloading:   0%|          | 0.00/862 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/494 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

{'score': 0.8113065361976624, 'start': 0, 'end': 10, 'answer': 'O rio Nilo'}


## Mounting Drive

In order to access the database using Colab, we used google drive

In [5]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)


Mounted at /content/drive/


## Reading dataset using Pandas Library



In [17]:
import pandas as pd
df = pd.read_csv('/content/drive/Shareddrives/Explainable QA/Base de dados/QAs/Blue_Amazon_QA_dataset.csv')
df.head()

,#,Source,book/wikipedia,Passage,Question,Answer,Passagem,Questão,Resposta
0,1,Campos Basin.txt,wikipedia,The Campos Basin is one of 12 coastal sediment...,How many coastal sedimentary basins are there ...,12.,A Bacia de Campos é uma das 12 bacias sediment...,Quantas bacias sedimentares costeiras existem ...,12.
1,2,Campos Basin.txt,wikipedia,The Campos Basin is one of 12 coastal sediment...,Why is the Campos Basin called that?,After the Campos dos Goytacazes city.,A Bacia de Campos é uma das 12 bacias sediment...,Por que a Bacia de Campos é chamada?,Depois da cidade de Campos dos Goytacazes.
2,3,Campos Basin.txt,wikipedia,The Campos Basin is one of 12 coastal sediment...,Where is located the onshore part of the Campo...,It's located near Rio de Janeiro.,A Bacia de Campos é uma das 12 bacias sediment...,Onde está localizado a parte onshore da Bacia ...,Está localizado perto do Rio de Janeiro.
3,4,Campos Basin.txt,wikipedia,The Campos Basin is one of 12 coastal sediment...,How big is the Campos Basin?,"It's about 115,000 square kilometres.",A Bacia de Campos é uma das 12 bacias sediment...,Quão grande é a bacia de Campos?,São cerca de 115.000 quilômetros quadrados.
4,5,Campos Basin.txt,wikipedia,The Campos Basin is one of 12 coastal sediment...,How old is the Campos Basin?,The Basin is 145 million old.,A Bacia de Campos é uma das 12 bacias sediment...,Quantos anos tem a Bacia de Campos?,A bacia é de 145 milhões de velha.


## Predicting responses across the entire dataset



In [8]:
import time
start_time = time.clock()

true_answers = []
gen_answers = []
passages = []
questions =[]
source = df.values.tolist()

# Loop over each question
for i in range(len(source)):
    #print every 10 questions
    if i%10==0:
      print(i)
    predictions = qa_pipeline({
    'context': source[i][6],
    'question': source[i][7]
    })
    # Saving to memory 
    passages.append(source[i][6])
    questions.append(source[i][7])
    gen_answers.append(predictions["answer"])
    true_answers.append(source[i][8])
print(time.clock() - start_time, "seconds")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
294.00177 seconds


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


## Metrics 

In this section,codes from three different metrics are used to evaluate the results: 


*   Macro average f1-score
*   Exact match
*   Rouge-l

It is important to note that this code is an adaptation from SQuAD dataset (Macro average f1-score and Exact match) and MS MARCO (Rouge-l)


In [9]:
# Imports
from rouge_score import rouge_scorer
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import SmoothingFunction
from nltk.metrics import f_measure
import nltk
import string
import re
import collections


# Supporting function
def get_tokens(sentence):
    tokens = word_tokenize(sentence)
    return tokens
def normalize_text(sentence):
    def white_space_fix(text):
        return " ".join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_punc(lower(sentence)))


# F1-score 
def f1_score_qa(text_true,text_pred):
    "squad"
    text_true = normalize_text(text_true)
    text_pred = normalize_text(text_pred)
    true_tokens = get_tokens(text_true)
    pred_tokens = get_tokens(text_pred)
    common = collections.Counter(true_tokens) & collections.Counter(pred_tokens)
    num_same = sum(common.values())
    if len(true_tokens) == 0 or len(pred_tokens) == 0:
        return int(true_tokens==pred_tokens),int(true_tokens==pred_tokens),int(true_tokens==pred_tokens)
    if num_same == 0:
        return 0,0,0
    precision = 1 * num_same / len(pred_tokens)
    recall = 1 * num_same / len(true_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1, precision, recall

# Exact Match
def em_qa(text_true,text_pred):
    return int(normalize_text(text_true) == normalize_text(text_pred))

# Rouge-l
def rouge_l_qa(text_true,text_pred):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    scores = scorer.score(text_true,text_pred)
    return scores["rougeL"].fmeasure

# Exact Match -> Loop over multiple instances
def em_qa_overall(text_true,text_pred):
    scores = 0
    for i in range(len(text_true)):
        scores += em_qa(text_true[i],text_pred[i])
    score = scores/len(text_true)
    return score

# F1-score -> Loop over multiple instances
def f1_score_qa_overall(text_true,text_pred):
    f = 0
    for i in range(len(text_true)):
        fi,pi,ri = f1_score_qa(text_true[i],text_pred[i])
        f+=fi
    f = f/len(text_true)
    return f

# Rouge-l -> Loop over multiple instances
def rouge_l_qa_overall(text_true,text_pred):
    scores = 0
    for i in range(len(text_true)):
        score = rouge_l_qa(text_true[i],text_pred[i])
        scores += score
    scores = scores/len(text_true)
    return scores


def scores(func,text_true,text_pred):
    if func=='rouge_l_qa':
        score_value = rouge_l_qa(text_true,text_pred)
    if func=='em_qa':
        score_value = em_qa(text_true,text_pred)
    if func=='f1_score_qa':
        score_value = f1_score_qa(text_true,text_pred)
    return (func, score_value)
        

def overall(func,text_true,text_pred):
    if func=='rouge_l_qa_overall':
        score_value = rouge_l_qa_overall(text_true,text_pred)
    if func=='em_qa_overall':
        score_value = em_qa_overall(text_true,text_pred)
    if func=='f1_score_qa_overall':
        score_value = f1_score_qa_overall(text_true,text_pred)
    return (func, score_value)

## Getting metrics

In [10]:
print("f1-score: " + str(overall("f1_score_qa_overall", true_answers, gen_answers)))
print("EM: " + str(overall('em_qa_overall', true_answers, gen_answers)))
print("Rouge-L: " + str(overall('rouge_l_qa_overall', true_answers, gen_answers)))

f1-score: ('f1_score_qa_overall', 0.5287518419575457)
EM: ('em_qa_overall', 0.22)
Rouge-L: ('rouge_l_qa_overall', 0.531298552111642)


## Saving answers to a dataframe

In [18]:
df = pd.DataFrame({'question':questions, 'passages': passages, 'true_answers': true_answers, "gen_answers": gen_answers})



## Getting metrics for each answer

In [12]:
from tqdm import tqdm
model_name = "test"
for _, row in tqdm(df.iterrows()):
    # F1-score
    _, f1_score = scores('f1_score_qa', row['gen_answers'], row['true_answers'])
    df.loc[df['question']==row['question'], '{}_f1'.format(model_name)] = round(f1_score[0], 2)

    # Rouge
    _, rouge_l = scores('rouge_l_qa', row['gen_answers'], row['true_answers'])
    rouge_l_f = rouge_l
    df.loc[df['question']==row['question'], '{}_rouge_l_f'.format(model_name)] = round(rouge_l_f, 2)

    # Exact Match
    _, em = scores('em_qa', row['gen_answers'], row['true_answers'])
    df.loc[df['question']==row['question'], '{}_em'.format(model_name)] = round(em, 2)

400it [00:01, 287.04it/s]


## Printing some of the results

In [16]:
df.head(15)

,question,passages,true_answers,gen_answers,test_f1,test_rouge_l_f,test_em
0,Quantas bacias sedimentares costeiras existem ...,A Bacia de Campos é uma das 12 bacias sediment...,12.,12,1.00,1.00,1.0
1,Por que a Bacia de Campos é chamada?,A Bacia de Campos é uma das 12 bacias sediment...,Depois da cidade de Campos dos Goytacazes.,uma das 12 bacias sedimentares costeiras,0.00,0.00,0.0
2,Onde está localizado a parte onshore da Bacia ...,A Bacia de Campos é uma das 12 bacias sediment...,Está localizado perto do Rio de Janeiro.,perto do Rio de Janeiro,0.83,0.83,0.0
3,Quão grande é a bacia de Campos?,A Bacia de Campos é uma das 12 bacias sediment...,São cerca de 115.000 quilômetros quadrados.,115.000 quilômetros quadrados,0.67,0.71,0.0
4,Quantos anos tem a Bacia de Campos?,A Bacia de Campos é uma das 12 bacias sediment...,A bacia é de 145 milhões de velha.,145-130 milhões de anos atrás,0.31,0.50,0.0
5,Qual é o significado da expressão brasileira p...,"Uma expressão famosa no Brasil é ""de Oiapoque ...","Significa ""do extremo sul até o extremo norte""...",do extremo sul até o extremo norte do país,0.95,0.95,0.0
6,Qual é o ponto mais ao norte do Brasil?,"Uma expressão famosa no Brasil é ""de Oiapoque ...",É o monte caburaí no estado de Roraima.,Monte Caburaí,0.40,0.44,0.0
7,Quão grande é a ilha de Santa Catarina?,"Florianópolis, a capital de Santa Catarina, es...","Tem uma área de 424,4 km².","424,4 km²",0.50,0.60,0.0
8,Qual estado brasileiro é responsável pelo lito...,"Piauí tem um litoral de apenas cerca de 60 km,...",O estado de Piauí.,Piauí,0.40,0.40,0.0
9,Onde estão localizados Lençóis Maranhenses?,"No Maranhão, algumas características geográfic...",Eles estão no Maranhão.,Maranhão,0.40,0.50,0.0


## Printing some of the correct answers

In [14]:
df.loc[df['test_f1']==1,:]

,question,passages,true_answers,gen_answers,test_f1,test_rouge_l_f,test_em
0,Quantas bacias sedimentares costeiras existem ...,A Bacia de Campos é uma das 12 bacias sediment...,12.,12,1.00,1.00,1.0
2,Onde está localizado a parte onshore da Bacia ...,A Bacia de Campos é uma das 12 bacias sediment...,Está localizado perto do Rio de Janeiro.,perto do Rio de Janeiro,0.83,0.83,0.0
5,Qual é o significado da expressão brasileira p...,"Uma expressão famosa no Brasil é ""de Oiapoque ...","Significa ""do extremo sul até o extremo norte""...",do extremo sul até o extremo norte do país,0.95,0.95,0.0
10,Quais são as praias de Amapá?,O litoral de Amapá é quase 600 km de comprimen...,"Existem apenas três praias: Fazendinha, Boca d...","Fazendinha, Boca do Inferno e Goiabal",0.75,0.71,0.0
16,Quão grande é a bacia de Santos?,A Bacia de Santos (Português: Bacia de Santos)...,aproximadamente 352.000 quilômetros quadrados ...,352.000 quilômetros quadrados,0.75,0.83,0.0
...,...,...,...,...,...,...,...
378,Qual é a maior ilha do arquipélago de Abrolhos?,Ilha de Santa Bárbara (Tirl. Santa Barbara Isl...,Ilha de Santa Bárbara,Ilha de Santa Bárbara,1.00,1.00,1.0
384,Quando a Ilha Grande e a Paraty inscreveram-se...,"Em 5 de julho de 2019, Ilha Grande e Paraty fo...",5 de julho de 2019.,5 de julho de 2019,1.00,1.00,1.0
390,Como foi formada a ilha de Monton de Trigo?,A ilha de Montão de Trigo (às vezes chamada Il...,De intensas atividades vulcânicas antigas.,intensas atividades vulcânicas antigas,0.89,0.91,0.0
393,Quais animais foram o foco de proteção do proj...,Embora o propósito inicial fosse proteger as t...,Tartarugas marinhas.,tartarugas marinhas,1.00,1.00,1.0


## Printing some of the incorrect answers

In [15]:
df.loc[df['test_f1']==0.,:]

,question,passages,true_answers,gen_answers,test_f1,test_rouge_l_f,test_em
1,Por que a Bacia de Campos é chamada?,A Bacia de Campos é uma das 12 bacias sediment...,Depois da cidade de Campos dos Goytacazes.,uma das 12 bacias sedimentares costeiras,0.0,0.00,0.0
23,Qual é a composição da formação de Itanhaém?,A formação de Itanhaém é de 517 metros (1.696 ...,"Consiste em shales cinzentos escuros, siltos e...",xícolas cinzentas escuras,0.0,0.00,0.0
30,Qual é o maior porto da América Latina?,"Uma vez foi considerado o ""porto de morte"" no ...",O porto de Santos.,Today,0.0,0.00,0.0
44,Quando o governo brasileiro passou uma lei par...,O porto de Santos foi originalmente fundado em...,No início dos anos 90.,1993,0.0,0.00,0.0
53,"O que significa o nome ""Ilha Compreda"" em port...","Ilha Compreida (Português para ""Long Island"")",Ilha Longa,Long Island,0.0,0.00,0.0
...,...,...,...,...,...,...,...
383,Há viagens de visão vistos em Ilha Grande?,O ecoturismo de pequena escala está sendo enco...,"Sim, existem.",várias trilhas e cachoeiras das montanhas da i...,0.0,0.00,0.0
386,Há quanto tempo a ilha de Monto de Trigo foi h...,"Nos últimos três séculos, a ilha foi permanent...",Por mais de 170 anos.,séculos,0.0,0.00,0.0
388,É verdade que os casamentos entre os membros d...,"Nos últimos três séculos, a ilha foi permanent...",sim.,Entre a mesma família,0.0,0.00,0.0
389,Onde está localizada a ilha de Montonão de Trigo?,"Nos últimos três séculos, a ilha foi permanent...",10km sul da costa de São Sebastião São Paulo.,norte do Brasil,0.0,0.00,0.0
